In [ ]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt


In [ ]:
with open("data/atom_transaction.json", "r") as f:
    transaction = json.load(f)


In [ ]:
from mlxtend.preprocessing import TransactionEncoder
transaction_values = [v for v in transaction.values()]
te = TransactionEncoder()
# te.fit([v for v in transaction_values])
te.fit(transaction_values)
te_ary = te.transform(transaction_values)
df_transaction = pd.DataFrame(te_ary, columns=te.columns_)


In [ ]:
from mlxtend.frequent_patterns import fpgrowth
df_freq_items = fpgrowth(df_transaction, min_support=0.1,
                         use_colnames=True)
df_freq_items.head(10)


In [ ]:
from mlxtend.frequent_patterns import association_rules
df_rules = association_rules(
    df_freq_items, metric="confidence", min_threshold=0.8)
df_rules["antecedent_len"] = df_rules["antecedents"].apply(
    lambda x: len(x))
df = df_rules[df_rules["antecedent_len"] == 1]
# 多いruleに並び直す．
df = df.sort_values(
    by="support", ascending=False).reset_index(drop=True)
df[["antecedents", "consequents", "antecedent support", 
    "confidence"]].head(10)


In [ ]:
# data 取得
df_atom_prop = pd.read_csv("data/atomicprop.csv", 
                           index_col=[0]).reset_index()
xlabel = "log10_electrical_resistivity"
ylabel = "log10_thermal_conductivity"
df_atom_prop = df_atom_prop[[xlabel,ylabel,"index"]].dropna()
df_atom_prop.reset_index(drop=True)
# show elements under the condition.
condition = "log10_electrical_resistivity<-6.9"
df_select = df_atom_prop.query(condition)
print(df_select["index"].values)
iselect = df_select.index.values
# set x and y
x = df_atom_prop[xlabel].values 
y = df_atom_prop[ylabel].values
from itemsetmining_misc import plot_atomicprop
plot_atomicprop(df_atom_prop, df_select, xlabel, ylabel)